In [1]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

In [2]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

TRAIN_DATASETS = [
    [
        "https://f000.backblazeb2.com/file/malay-dataset/paraphrase/train-paraphrase.tar.gz", 
        ("train/before.txt",
         "train/after.txt")
    ]
]

TEST_DATASETS = [
    [
        "https://f000.backblazeb2.com/file/malay-dataset/paraphrase/test-paraphrase.tar.gz", 
        ("test/before.txt",
         "test/after.txt")
    ]
]

In [3]:
@registry.register_problem
class Paraphrase32k(translate.TranslateProblem):
    """En-de translation trained on WMT corpus."""

    @property
    def additional_training_datasets(self):
        """Allow subclasses to add training datasets."""
        return []

    def source_data_files(self, dataset_split):
        train = dataset_split == problem.DatasetSplit.TRAIN
        train_datasets = TRAIN_DATASETS + self.additional_training_datasets
        return train_datasets if train else TEST_DATASETS

In [4]:
PROBLEM = 'paraphrase32k'
problem = problems.problem(PROBLEM)

In [6]:
import tensorflow as tf
import os

vocab_file = "t2t-tiny/data/vocab.paraphrase32k.32768.subwords"
ckpt_path = tf.train.latest_checkpoint(os.path.join('t2t-tiny/train'))
vocab_file, ckpt_path

('t2t-tiny/data/vocab.paraphrase32k.32768.subwords',
 't2t-tiny/train/model.ckpt-100000')

In [7]:
from t import text_encoder

encoder = text_encoder.SubwordTextEncoder(vocab_file)

In [8]:
class Model:
    def __init__(self, HPARAMS = "transformer_tiny", DATA_DIR = 't2t-tiny/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 2 * tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        print(features)
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=5, 
                top_beams=1, alpha=1.0)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


{'inputs': <tf.Tensor 'ExpandDims_1:0' shape=(?, ?, 1, 1) dtype=int32>, 'targets': <tf.Tensor 'ExpandDims_3:0' shape=(?, ?, 1, 1) dtype=int32>, 'target_space_id': <tf.Tensor 'Const_1:0' shape=() dtype=int32>}



INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


...

INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_26357_128.top


INFO:tensorflow:Transforming body output with symbol_modality_26357_128.top


INFO:tensorflow:Restoring parameters from t2t-tiny/train/model.ckpt-100000


INFO:tensorflow:Restoring parameters from t2t-tiny/train/model.ckpt-100000


In [9]:
string = "Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib."
encoded = encoder.encode(string) + [1]
encoded

[25899,
 6,
 31,
 2876,
 26278,
 26289,
 26280,
 11484,
 1067,
 898,
 2,
 73,
 373,
 26281,
 26289,
 26280,
 26287,
 26289,
 26282,
 26293,
 3683,
 65,
 3346,
 2937,
 136,
 238,
 26311,
 26293,
 26284,
 26285,
 1619,
 3,
 1]

In [10]:
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})

In [11]:
encoder.decode(f[0])

'Dia juga seorang saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib.<EOS>'

In [12]:
encoder.decode(b[0])

'Dia juga seorang saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib.<EOS>'

In [13]:
%%time

sess.run(model.fast_result, feed_dict = {model.X: [encoded, encoded]})

CPU times: user 974 ms, sys: 70.9 ms, total: 1.05 s
Wall time: 669 ms


array([[   26,    31,    53,  2876, 26278, 26289, 26280, 11484,  1067,
          898,     2,    73,   373, 26281, 26289, 26280, 26287, 26289,
        26282, 26293,  3683,    65,  3346,  2937,   136,   238, 26311,
        26293, 26284, 26285,  1619,     3,     1],
       [   26,    31,    53,  2876, 26278, 26289, 26280, 11484,  1067,
          898,     2,    73,   373, 26281, 26289, 26280, 26287, 26289,
        26282, 26293,  3683,    65,  3346,  2937,   136,   238, 26311,
        26293, 26284, 26285,  1619,     3,     1]])

In [14]:
batch_size = 32

path = 't2t/tmp/test'

with open(os.path.join(path, 'before.txt')) as fopen:
    left = fopen.read().split('\n')
    
with open(os.path.join(path, 'after.txt')) as fopen:
    right = fopen.read().split('\n')
    
len(left), len(right)

(5872, 5872)

In [15]:
left[0], right[0]

('Stesen Williamstown terletak di laluan Williamstown Utara di Victoria, Australia.',
 'Stesen kereta api Williamstown terletak di garisan Williamstown Utara di Victoria, Australia.')

In [16]:
p = sess.run(model.fast_result, feed_dict = {model.X: [encoder.encode(left[0]) + [1]]}).tolist()
results = []
for row in p:
    results.append([i for i in row if i not in [0, 1]])
results

[[468, 8765, 69, 5, 462, 8765, 345, 5, 1546, 2, 229, 3]]

In [17]:
from tensor2tensor.utils import bleu_hook

In [18]:
bleu_hook.compute_bleu(reference_corpus = [encoder.encode(right[0])], 
                       translation_corpus = results)

0.5496509

In [19]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [20]:
from tqdm import tqdm

results = []
for i in tqdm(range(0, len(left), batch_size)):
    index = min(i + batch_size, len(left))
    x = left[i: index]
    encoded = [encoder.encode(l) + [1] for l in x]
    batch_x = pad_sequences(encoded, padding='post')
    
    p = sess.run(model.fast_result, feed_dict = {model.X: batch_x}).tolist()
    result = []
    for row in p:
        result.append([i for i in row if i not in [0, 1]])
    results.extend(result)

100%|██████████| 184/184 [02:37<00:00,  1.17it/s]


In [21]:
rights = [encoder.encode(r) for r in right[:len(results)]]
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.5941754

In [22]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-tiny/model.ckpt')

'transformer-tiny/model.ckpt'

In [23]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attenti

In [24]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [25]:
freeze_graph('transformer-tiny', strings)

INFO:tensorflow:Restoring parameters from transformer-tiny/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-tiny/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 81 variables.


INFO:tensorflow:Froze 81 variables.


INFO:tensorflow:Converted 81 variables to const ops.


INFO:tensorflow:Converted 81 variables to const ops.


4430 ops in the final graph.


In [26]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [27]:
g = load_graph('transformer-tiny/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [28]:
string = "Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib."
encoded = encoder.encode(string) + [1]
g, b = test_sess.run([greedy, beam], feed_dict = {x:[encoded]})

In [29]:
encoder.decode(g[0])

'Dia juga seorang saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib.<EOS>'

In [30]:
encoder.decode(b[0])

'Dia juga seorang saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib.<EOS>'